# Lab 9: Tracing and Observability for AI Agents

Build **production-ready observability** for your AI agents using OpenTelemetry, Azure Monitor, and Microsoft Foundry's tracing capabilities.

## Learning Objectives

By the end of this lab, you will:
1. Understand **tracing concepts** (traces, spans, attributes, semantic conventions)
2. Set up **Azure Monitor** integration for trace collection
3. Instrument your **AI agents** with OpenTelemetry tracing
4. Create **custom spans** for your own functions
5. View and analyze traces in the **Foundry portal**
6. Implement **best practices** for production observability

## Prerequisites

| Requirement | Setup |
|------------|-------|
| Labs 1-8 completed | Agent patterns, evaluations |
| Python environment | Run `uv sync --all-extras --dev` from project root |
| Azure CLI logged in | Run `az login` in terminal |
| Azure AI Foundry Project | [Create at ai.azure.com](https://ai.azure.com) |
| Application Insights | Connected to your Foundry project |

## Why Observability Matters

> 🎯 **The Goal**: Understand exactly what your AI agent is doing, why it made certain decisions, and where issues occur.

| Without Observability | With Observability |
|----------------------|--------------------|
| "The agent gave a wrong answer" | See exact LLM inputs/outputs at each step |
| "It's slow sometimes" | Identify which tool call or API is the bottleneck |
| "Something failed" | Pinpoint the exact span where the error occurred |
| "How much did that cost?" | Track token usage per request |

## Environment Variables

```bash
# Required in .env file
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

## Tracing Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                     AI Agent Observability Pipeline                          │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌─────────────┐     ┌─────────────────────────────────────────────────┐   │
│   │   AI Agent  │────▶│              OpenTelemetry SDK                  │   │
│   │  (Your App) │     │  • Tracer Provider                              │   │
│   └─────────────┘     │  • Span Processors                              │   │
│                       │  • Exporters                                    │   │
│                       └─────────────────────────────────────────────────┘   │
│                                          │                                  │
│                    ┌─────────────────────┴─────────────────────┐            │
│                    ▼                                           ▼            │
│   ┌────────────────────────────┐       ┌────────────────────────────┐      │
│   │     Azure Monitor          │       │     Console / Local        │      │
│   │  (Application Insights)    │       │    (Aspire Dashboard)      │      │
│   │  • Production traces       │       │  • Development debugging   │      │
│   │  • Foundry portal view     │       │  • Quick iteration         │      │
│   └────────────────────────────┘       └────────────────────────────┘      │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Key Concepts

| Concept | Description | Example |
|---------|-------------|--------|
| **Trace** | End-to-end journey of a request | User query → Agent response |
| **Span** | Single operation within a trace | LLM call, tool execution |
| **Attributes** | Key-value metadata on spans | `model.name`, `token.count` |
| **Semantic Conventions** | Standardized attribute names | OpenTelemetry GenAI conventions |
| **Exporter** | Sends trace data to backend | Azure Monitor, Console |

## Span Hierarchy Example

```
Trace: user_request_12345
│
├── Span: agent_run (root)
│   ├── Span: llm_call_1 (planning)
│   │   └── attributes: model=gpt-4o, tokens=150
│   │
│   ├── Span: tool_call (fetch_weather)
│   │   ├── attributes: tool=fetch_weather, args={location: Seattle}
│   │   └── Span: http_request (weather API)
│   │
│   └── Span: llm_call_2 (final response)
│       └── attributes: model=gpt-4o, tokens=200
```

> 📚 **Documentation**: [Trace and observe AI agents](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-observe-agents)

# Part 1: Setup and Environment

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

# Ensure Azure CLI is in PATH
for p in ["/opt/homebrew/bin", "/usr/local/bin"]:
    if p not in os.environ.get("PATH", ""):
        os.environ["PATH"] = p + ":" + os.environ.get("PATH", "")

# Load .env
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded {env_path.resolve()}")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"✅ Endpoint: {endpoint[:50]}..." if endpoint else "❌ Missing AZURE_AI_PROJECT_ENDPOINT")
print(f"✅ Model: {model}")

✅ Loaded /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.env
✅ Endpoint: https://nw-fy-26.services.ai.azure.com/api/project...
✅ Model: gpt-4.1


In [2]:
# Install tracing packages
# Run in terminal: uv pip install azure-monitor-opentelemetry opentelemetry-sdk azure-core-tracing-opentelemetry

# Core imports
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# OpenTelemetry imports
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter

print("✅ All imports successful")

✅ All imports successful


# Part 2: Enable Content Recording

By default, Azure AI tracing does **not** record the actual content of messages (for privacy). To see full inputs/outputs, enable content recording:

> ⚠️ **Warning**: Only enable in development. Content may contain personal data.

In [3]:
# Enable content recording for development
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

print("✅ Content recording enabled")
print("   Trace data will include full message content")
print("\n⚠️  Remember to disable in production if content contains PII")

✅ Content recording enabled
   Trace data will include full message content

⚠️  Remember to disable in production if content contains PII


# Part 3: Console Tracing (Local Development)

For quick iteration during development, trace to the console. This doesn't require any cloud resources.

In [4]:
# Configure console tracing
from azure.core.settings import settings
settings.tracing_implementation = "opentelemetry"

# Set up tracer provider with console exporter
console_exporter = ConsoleSpanExporter()
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(console_exporter))
trace.set_tracer_provider(tracer_provider)

# Get a tracer for our application
tracer = trace.get_tracer("lab9-tracing-demo")

print("✅ Console tracing configured")
print("   Spans will be printed to stdout")

✅ Console tracing configured
   Spans will be printed to stdout


In [5]:
# Test basic tracing with a simple span
print("🔍 Testing basic span creation...\n")

with tracer.start_as_current_span("test-span") as span:
    span.set_attribute("test.attribute", "hello-world")
    span.set_attribute("lab.number", 9)
    print("   Inside the span - doing some work...")

print("\n✅ Span completed - check console output above for trace data")

🔍 Testing basic span creation...

   Inside the span - doing some work...
{
    "name": "test-span",
    "context": {
        "trace_id": "0x02be52d1acc3a716467cc74ebb38aecd",
        "span_id": "0xa790de2fcb79218b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2026-02-01T20:35:15.716199Z",
    "end_time": "2026-02-01T20:35:15.716239Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "test.attribute": "hello-world",
        "lab.number": 9
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.39.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}

✅ Span completed - check console output above for trace data


# Part 4: Create an Agent with Tracing

Now let's create an AI agent and trace its execution. We'll use `agent_framework` and wrap the execution in spans.

In [6]:
# Import agent framework
from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import AzureCliCredential

# Define tools for the agent
def fetch_weather(location: str) -> dict:
    """Fetches current weather for a location."""
    weather_data = {
        "Seattle": {"temp": "14°C", "condition": "Rainy"},
        "New York": {"temp": "22°C", "condition": "Sunny"},
        "London": {"temp": "16°C", "condition": "Cloudy"},
    }
    return weather_data.get(location, {"temp": "20°C", "condition": "Unknown"})

def calculate_tip(bill_amount: float, tip_percent: float = 18.0) -> dict:
    """Calculates tip amount for a bill."""
    tip = bill_amount * (tip_percent / 100)
    return {"tip_amount": round(tip, 2), "total": round(bill_amount + tip, 2)}

print("✅ Tools defined: fetch_weather, calculate_tip")

✅ Tools defined: fetch_weather, calculate_tip


In [7]:
# Run agent with tracing
async def run_agent_with_tracing(query: str):
    """Run an agent and trace the entire execution."""
    
    # Create a root span for the entire agent run
    with tracer.start_as_current_span("agent_run") as root_span:
        root_span.set_attribute("user.query", query)
        root_span.set_attribute("agent.name", "TracingDemoAgent")
        
        async with AzureCliCredential() as credential:
            async with AzureAIAgentsProvider(credential=credential) as provider:
                
                # Span for agent creation
                with tracer.start_as_current_span("create_agent") as create_span:
                    create_span.set_attribute("agent.instructions", "You are a helpful assistant.")
                    
                    agent = await provider.create_agent(
                        name="TracingDemoAgent",
                        instructions="You are a helpful assistant. Use tools when appropriate.",
                        tools=[fetch_weather, calculate_tip],
                    )
                    create_span.set_attribute("agent.tools", ["fetch_weather", "calculate_tip"])
                
                # Span for agent execution
                with tracer.start_as_current_span("agent_execute") as exec_span:
                    response = await agent.run(query)
                    response_str = str(response)
                    
                    exec_span.set_attribute("response.length", len(response_str))
                    exec_span.set_attribute("response.preview", response_str[:100])
        
        root_span.set_attribute("status", "success")
        return response_str

print("✅ Tracing function defined")

✅ Tracing function defined


In [8]:
# Execute the agent with tracing
print("🤖 Running agent with tracing enabled...\n")
print("=" * 70)

result = await run_agent_with_tracing("What's the weather in Seattle?")

print("=" * 70)
print(f"\n📝 Agent Response: {result[:200]}...")

🤖 Running agent with tracing enabled...

{
    "name": "POST",
    "context": {
        "trace_id": "0x33e6b23ea98bbf4ce88456c4ae9af9e7",
        "span_id": "0x67880bb1bd57c6c0",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": "0x83db4d260836d682",
    "start_time": "2026-02-01T20:35:17.926404Z",
    "end_time": "2026-02-01T20:35:18.812668Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "component": "http",
        "http.method": "POST",
        "http.url": "https://nw-fy-26.services.ai.azure.com/api/projects/dev/assistants?api-version=2025-05-15-preview",
        "net.peer.name": "nw-fy-26.services.ai.azure.com",
        "user_agent.original": "agent-framework-python/1.0.0b260130 azsdk-python-ai-agents/1.2.0b5 Python/3.11.14 (macOS-26.2-arm64-arm-64bit)",
        "http.status_code": 200,
        "az.client_request_id": "8503ba2c-ffad-11f0-855b-e5dfa0a8dc26"
    },
    "events": [],
    "links": [],
    "resource":

# Part 5: Azure Monitor Tracing (Production)

For production, send traces to **Azure Monitor Application Insights**. This enables:
- Viewing traces in the Foundry portal
- Long-term storage and analysis
- Alerting on errors or latency
- Correlation with other Azure services

## Setup Steps

1. Navigate to **Tracing** in the Foundry portal left navigation
2. Create or connect an **Application Insights** resource
3. Copy the connection string

In [9]:
# Configure Azure Monitor tracing
# Install if needed: uv pip install azure-monitor-opentelemetry

try:
    from azure.monitor.opentelemetry import configure_azure_monitor
    AZURE_MONITOR_AVAILABLE = True
except ImportError:
    AZURE_MONITOR_AVAILABLE = False
    print("⚠️ azure-monitor-opentelemetry not installed")
    print("   Install with: uv pip install azure-monitor-opentelemetry")

if AZURE_MONITOR_AVAILABLE:
    # Connect to your Foundry project
    with DefaultAzureCredential() as credential:
        project_client = AIProjectClient(
            endpoint=endpoint,
            credential=credential,
        )
        
        # Get Application Insights connection string from project
        try:
            connection_string = project_client.telemetry.get_application_insights_connection_string()
            print(f"✅ Application Insights connected")
            print(f"   Connection string: {connection_string[:50]}...")
            
            # Configure Azure Monitor
            configure_azure_monitor(connection_string=connection_string)
            print("\n✅ Azure Monitor tracing enabled")
            print("   Traces will appear in Foundry portal under 'Tracing'")
            
        except Exception as e:
            print(f"⚠️ Could not connect to Application Insights: {e}")
            print("   Make sure Application Insights is connected to your Foundry project")

{
    "name": "GET",
    "context": {
        "trace_id": "0xdfe5442bc3fb34e652816591a61c4722",
        "span_id": "0xd6725310827c055b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": "0x5991c8fc48f1580a",
    "start_time": "2026-02-01T20:35:25.630760Z",
    "end_time": "2026-02-01T20:35:25.632686Z",
    "status": {
        "status_code": "ERROR",
        "description": "azure.core.exceptions.ServiceRequestError: HTTPConnection(host='169.254.169.254', port=80): Failed to establish a new connection: [Errno 45] Operation not supported"
    },
    "attributes": {
        "component": "http",
        "http.method": "GET",
        "http.url": "http://169.254.169.254/metadata/identity/oauth2/token?api-version=2018-02-01&resource=https://ai.azure.com",
        "net.peer.name": "169.254.169.254",
        "user_agent.original": "azsdk-python-identity/1.26.0b1 Python/3.11.14 (macOS-26.2-arm64-arm-64bit)",
        "http.status_code": 504,
        "error.type": 

In [10]:
# Run agent with Azure Monitor tracing
print("🤖 Running agent with Azure Monitor tracing...\n")

# Get a new tracer (will use Azure Monitor exporter)
az_tracer = trace.get_tracer("lab9-azure-monitor-demo")

async def run_traced_agent(query: str):
    with az_tracer.start_as_current_span("traced_agent_session") as span:
        span.set_attribute("session.id", "lab9-demo-001")
        span.set_attribute("user.query", query)
        
        async with AzureCliCredential() as credential:
            async with AzureAIAgentsProvider(credential=credential) as provider:
                agent = await provider.create_agent(
                    name="AzureMonitorAgent",
                    instructions="You are a helpful assistant.",
                    tools=[fetch_weather, calculate_tip],
                )
                response = await agent.run(query)
                span.set_attribute("response.success", True)
                return str(response)

result = await run_traced_agent("Calculate a 20% tip on a $85 bill")
print(f"\n📝 Response: {result}")
print("\n📎 View trace in Foundry portal: Project → Tracing")

🤖 Running agent with Azure Monitor tracing...

{
    "name": "POST",
    "context": {
        "trace_id": "0x4d1488866af2ee89406fbb918cb58074",
        "span_id": "0x36b7d57ff32d5a68",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": "0x67e326e6b35a400d",
    "start_time": "2026-02-01T20:35:26.793178Z",
    "end_time": "2026-02-01T20:35:27.746244Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "component": "http",
        "http.method": "POST",
        "http.url": "https://nw-fy-26.services.ai.azure.com/api/projects/dev/assistants?api-version=2025-05-15-preview",
        "net.peer.name": "nw-fy-26.services.ai.azure.com",
        "user_agent.original": "agent-framework-python/1.0.0b260130 azsdk-python-ai-agents/1.2.0b5 Python/3.11.14 (macOS-26.2-arm64-arm-64bit)",
        "http.status_code": 200,
        "az.client_request_id": "8a63115c-ffad-11f0-855b-e5dfa0a8dc26"
    },
    "events": [],
    "links": [],
    "reso

# Part 6: Custom Function Tracing

Trace your own functions to get visibility into business logic, data processing, or external API calls.

In [11]:
from opentelemetry.trace import SpanKind
import time

# Custom traced function - simulates data processing
def process_data(data: list) -> dict:
    """Process data with full tracing."""
    with tracer.start_as_current_span("process_data", kind=SpanKind.INTERNAL) as span:
        span.set_attribute("input.count", len(data))
        span.set_attribute("input.type", type(data).__name__)
        
        # Simulate processing steps
        with tracer.start_as_current_span("validate_data") as validate_span:
            valid_items = [x for x in data if x is not None]
            validate_span.set_attribute("valid.count", len(valid_items))
            validate_span.set_attribute("invalid.count", len(data) - len(valid_items))
        
        with tracer.start_as_current_span("transform_data") as transform_span:
            # Simulate transformation
            time.sleep(0.1)  # Simulate work
            result = {"sum": sum(valid_items), "count": len(valid_items), "avg": sum(valid_items) / len(valid_items)}
            transform_span.set_attribute("result.sum", result["sum"])
        
        span.set_attribute("output.keys", list(result.keys()))
        return result

# Custom traced function - simulates external API call
def call_external_api(endpoint: str, payload: dict) -> dict:
    """Make an external API call with tracing."""
    with tracer.start_as_current_span("external_api_call", kind=SpanKind.CLIENT) as span:
        span.set_attribute("http.method", "POST")
        span.set_attribute("http.url", endpoint)
        span.set_attribute("request.payload_size", len(str(payload)))
        
        # Simulate API call
        time.sleep(0.2)
        response = {"status": "success", "data": payload}
        
        span.set_attribute("http.status_code", 200)
        span.set_attribute("response.size", len(str(response)))
        return response

print("✅ Custom traced functions defined")

✅ Custom traced functions defined


In [12]:
# Run custom traced functions
print("🔍 Running custom traced functions...\n")
print("=" * 70)

# Test data processing
with tracer.start_as_current_span("data_pipeline") as pipeline_span:
    pipeline_span.set_attribute("pipeline.name", "demo-pipeline")
    
    data_result = process_data([1, 2, 3, None, 5, 6, None, 8])
    api_result = call_external_api("https://api.example.com/data", data_result)

print("=" * 70)
print(f"\n📊 Data Result: {data_result}")
print(f"📡 API Result: {api_result}")

🔍 Running custom traced functions...

{
    "name": "validate_data",
    "context": {
        "trace_id": "0xbc773e0e68516eab8c033eb2399bc702",
        "span_id": "0xa6342e1c6223de07",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x262255eeb41547f6",
    "start_time": "2026-02-01T20:35:34.439982Z",
    "end_time": "2026-02-01T20:35:34.439988Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "valid.count": 6,
        "invalid.count": 2
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.39.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "transform_data",
    "context": {
        "trace_id": "0xbc773e0e68516eab8c033eb2399bc702",
        "span_id": "0x1498c1b852edc4c8",
        "trace_st

# Part 7: Agent Instrumentation with AIAgentsInstrumentor

For automatic instrumentation of Azure AI Agents, use the built-in `AIAgentsInstrumentor`.

In [13]:
# Enable automatic agent instrumentation
try:
    from azure.ai.agents.telemetry import AIAgentsInstrumentor
    
    # Instrument the Azure AI Agents SDK
    AIAgentsInstrumentor().instrument()
    
    print("✅ AIAgentsInstrumentor enabled")
    print("   All agent operations will be automatically traced")
    print("   Includes: agent creation, message handling, tool calls, runs")
    
except ImportError:
    print("⚠️ AIAgentsInstrumentor not available")
    print("   Install with: pip install azure-ai-projects[agents]")
    print("   Manual tracing will still work")

✅ AIAgentsInstrumentor enabled
   All agent operations will be automatically traced
   Includes: agent creation, message handling, tool calls, runs


# Part 8: Viewing Traces in Foundry Portal

## Steps to View Traces

1. **Navigate to Foundry Portal**: [ai.azure.com](https://ai.azure.com)
2. **Select your project**
3. **Click "Tracing"** in the left navigation
4. **Filter traces** by time range, service name, or status
5. **Click a trace** to see the span hierarchy

## What You'll See

```
┌─────────────────────────────────────────────────────────────────────────────┐
│  Trace: traced_agent_session                           Duration: 2.3s      │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ├── create_agent                                      150ms               │
│  │   └── attributes: agent.name=AzureMonitorAgent                          │
│  │                                                                          │
│  ├── llm_call (planning)                               800ms               │
│  │   └── attributes: model=gpt-4o, tokens.input=45                         │
│  │                                                                          │
│  ├── tool_call (calculate_tip)                         50ms                │
│  │   └── attributes: tool.args={bill: 85, percent: 20}                     │
│  │                                                                          │
│  └── llm_call (response)                               1200ms              │
│      └── attributes: model=gpt-4o, tokens.output=120                       │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Azure Monitor Query (KQL)

```sql
-- Find all traces from your service
traces
| where cloud_RoleName == "lab9-azure-monitor-demo"
| order by timestamp desc
| take 100
```

# Summary and Best Practices

## Tracing Best Practices

| Practice | Why | Example |
|----------|-----|--------|
| **Use consistent span names** | Easier to filter and analyze | `agent_run`, `tool_call`, `llm_request` |
| **Add meaningful attributes** | Context for debugging | `user.id`, `model.name`, `token.count` |
| **Use semantic conventions** | Interoperability | OpenTelemetry GenAI conventions |
| **Redact sensitive data** | Security & compliance | Don't log passwords, PII |
| **Set service name** | Multi-app distinction | `OTEL_SERVICE_NAME` env var |
| **Enable content recording** | Debug, but only in dev | `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` |

## Span Attribute Conventions

| Category | Attribute | Description |
|----------|-----------|-------------|
| **Agent** | `agent.name` | Agent identifier |
| **Agent** | `agent.instructions` | System prompt (redact if sensitive) |
| **LLM** | `gen_ai.model` | Model name |
| **LLM** | `gen_ai.usage.input_tokens` | Tokens in prompt |
| **LLM** | `gen_ai.usage.output_tokens` | Tokens in response |
| **Tool** | `tool.name` | Tool function name |
| **Tool** | `tool.call.arguments` | Tool input args |
| **Tool** | `tool.call.results` | Tool output |
| **Error** | `error.type` | Exception class |
| **Error** | `error.message` | Error description |

## Integration Summary

| Framework | Tracing Support |
|-----------|----------------|
| **Azure AI Agents** | Native with `AIAgentsInstrumentor` |
| **Semantic Kernel** | Native integration |
| **LangChain/LangGraph** | `langchain-azure-ai` tracer |
| **OpenAI Agents SDK** | `opentelemetry-instrumentation-openai-agents` |
| **Custom code** | Manual OpenTelemetry spans |

## Further Reading & Resources

### Microsoft Documentation
- [Trace and observe AI agents](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-observe-agents)
- [Observability basics](https://learn.microsoft.com/azure/ai-foundry/concepts/observability)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)

### OpenTelemetry
- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/languages/python/)
- [Semantic Conventions for GenAI](https://opentelemetry.io/docs/specs/semconv/gen-ai/)
- [Trace Context (W3C)](https://www.w3.org/TR/trace-context/)

### Tools
- [AI Toolkit for VS Code](https://marketplace.visualstudio.com/items?itemName=ms-windows-ai-studio.windows-ai-studio)
- [Aspire Dashboard](https://learn.microsoft.com/dotnet/aspire/fundamentals/dashboard/overview)